In [34]:
import torch
import torch.nn as nn
import pandas as pd
import sys
sys.path.append('../')
from scripts.tools import *
from sklearn.metrics import accuracy_score
from pathlib import Path
from scripts.tools import _train_test_split
from scripts.tools import data_loader
import json

In [35]:
PATH = '../meta/cuisines.json'

with open(PATH,'r',encoding='utf-8') as f:
    raw = f.read()
    data = json.loads(raw)

ground_thruts = data['cuisines']

In [36]:
len(ground_thruts)

20

In [37]:
base_dir = '/home/ubuntu/random_forest/recipe_classifier'
PATH = f'{base_dir}/models/NeuralNetwork.pth'

In [38]:
# df = pd.read_csv('../data/train.csv')
df = pd.read_csv('../data/cleaned_validation.csv')

In [39]:
labels = list(df.columns)[:-1]
X = df[labels].values
Y = df['cuisine'].values
cuisines = list(set(Y))
cuisines.sort()


In [40]:
X,Y = remove_invalid(X,Y)

In [41]:
ratios = get_class_ratio(Y,_print=True)

* italian 7838
* mexican 6438
* southern_us 4320
* indian 3003
* chinese 2673
* french 2646
* thai 1539
* japanese 1423
* greek 1175
* spanish 989
* korean 830
* vietnamese 824
* moroccan 821
* british 804
* filipino 755
* irish 667
* jamaican 526
* russian 488
* brazilian 467


In [42]:
Y = [cuisines.index(y) for y in Y]

In [43]:
_, X_test, _, Y_test = _train_test_split(X, Y,test_size=1)

In [44]:

X_test = torch.tensor(X_test,dtype=torch.float32)
Y_test = torch.tensor(Y_test)

In [45]:
class NeurelNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeurelNetwork, self).__init__()

        self.l1 = nn.Linear(input_size, 125)
        self.l2 = nn.Linear(125, output_size)

    def forward(self, X):
        out = torch.rrelu(self.l1(X))
        out = torch.rrelu(self.l2(out))

        return out

In [46]:
n_features = len(X[0])
n_outputs = len(ground_thruts)
model = NeurelNetwork(n_features, n_outputs)
model.load_state_dict(torch.load(PATH))
# model.eval()


<All keys matched successfully>

In [47]:
with torch.no_grad():
    Y_pred = model(X_test)

    # l = loss(Y_pred, Y_test)

    _, predicted = torch.max(Y_pred.data, 1)

    print(predicted.shape)
    print(Y_test.shape)

    score = accuracy_score(predicted, Y_test)
    print(f'score: {score}')

torch.Size([38227])
torch.Size([38227])
score: 0.7263452533549586
